In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from datasets import load_dataset
DATASET_CACHE = os.getenv('DATASET_CACHE', '/tmp/dataset_cache')
TASKS_TO_PROCESS = 100

/scratch/git/tunix/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("R2E-Gym/R2E-Gym-V1", split="train", cache_dir=DATASET_CACHE, num_proc=32)
entries = []
unique_images = set()
for i, entry in enumerate(dataset):
  if "docker_image" in entry:
    unique_images.add(entry["docker_image"])
    entries.append(entry)
  if i >= TASKS_TO_PROCESS - 1:
    break
unique_images = list(unique_images)
print(f"Found {len(unique_images)} unique Docker images to download")
IDS = [f"task-{i}" for i in range(len(entries))]

Found 100 unique Docker images to download


In [4]:
import os

os.environ["KUBECONFIG"] = "~/.kube/config"

from kubernetes import client, config
config.load_kube_config()
k8s_client = client.CoreV1Api()
# k8s_client.list_namespace(timeout_seconds=5)

In [16]:
import r2egym

r2egym.__file__
from r2egym.agenthub.runtime.docker import DockerRuntime
from r2egym.agenthub.utils.log import get_logger

In [18]:
runtime = DockerRuntime(ds=entries[0], command=["/bin/bash", "-l"], logger=get_logger(), backend="kubernetes", id=IDS[0])

KeyboardInterrupt: 

In [ ]:
DOCKER_PATH = "/root/.venv/bin:/root/.local/bin:/root/.cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
pod_name = "tsbao-test-pod"
docker_image = entries[0]["docker_image"]
command = "/bin/bash"

env_vars = {"PATH": DOCKER_PATH}
env_spec = [{"name": k, "value": str(v)} for k, v in env_vars.items()]
pod_body = {
    "apiVersion": "v1",
    "kind": "Pod",
    "metadata": {"name": pod_name},
    "spec": {
        "restartPolicy": "Never",
        "containers": [
            {
                "name": pod_name,
                "image": docker_image,
                "command": ["/bin/sh", "-c"],
                "args": [command] if isinstance(command, str) else command,
                "stdin": True,
                "tty": True,
                "env": env_spec,
                "resources": {
                    "requests": {"cpu": "1", "memory": "1Gi"},
                },
            }
        ],
        "imagePullSecrets": [{"name": "dockerhub-pro"}],
        "nodeSelector": {"cloud.google.com/gke-nodepool": "tsbao-gvisor-cpu-pool"},
        "tolerations": [
            {
                "key": "node.kubernetes.io/disk-pressure",
                "operator": "Exists",
                "effect": "NoExecute",
                "tolerationSeconds": 10800
            }
        ],
    },
}

pod = k8s_client.create_namespaced_pod(
    namespace="default", body=pod_body, _request_timeout=60,
)

In [5]:
# k8s_client.list_namespaced_pod(namespace="default")
pod_name = "tsbao-test-pod"
pod = k8s_client.read_namespaced_pod(name=pod_name, namespace="default")
pod.status.phase



'Pending'